In [ ]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
np.random.seed(42)  

# (b) and (c) parts
def sampling(n, beta):
    constants = np.ones((n,1))
    beta0 = beta[0]
    beta1 = beta[1]
    covariate = np.random.randn(n,1)
    hazard_rate = np.exp(beta0 + beta1*covariate).flatten()
    independent = np.hstack((intercept, covariate))
    
    U = np.random.uniform(0, 1, n)
    
    dependent = -np.log(1 - U)/hazard_rate
    
    return independent, dependent

n, beta = 500, [1, 1]
X, Y = sampling(n, beta)

In [ ]:
# (e)
def log_likelihood(beta, independent, dependent):
    
    # sums up individual likelihood contributions for all 500 observations
    return  np.sum(independent@beta - np.exp(independent@beta)*dependent)

def gradient(beta, independent, dependent):
    
    partial0 = 1 - np.exp(independent@beta)*dependent
    partial1 = independent[1] - np.exp(independent@beta)*independent[1]*dependent
    
    return np.array([partial0, partial1])
    
def hessian(beta, independent, dependent):
    partial00 = -np.exp(independent@beta)*dependent
    partial11 = -independent[1]**2*np.exp(independent@beta)*dependent
    partial10 = -independent[1]*np.exp(independent@beta)*dependent
    
    return np.array([partial00, partial10, partial10, partial11])

def outer_prod(beta, independent, dependent):
    grad_vector = gradient(beta, independent, dependent).reshape(-1, 1)
    return grad_vector@grad_vector.T

def summed_matrices(beta): # for computing newton raphson and bhcube updates
    sum_grad = [0, 0]
    sum_hessian = [0, 0, 0, 0]
    sum_outer = [[0, 0],[0, 0]]
    
    for i in range(len(X)):
        grad = gradient(beta, X[i], Y[i])
        sum_grad += grad

        hessian_matrix = hessian(beta, X[i], Y[i])
        sum_hessian += hessian_matrix

        outer_matrix = outer_prod(beta, X[i], Y[i])
        sum_outer += outer_matrix
    
    return sum_grad, sum_hessian, sum_outer

def newton_update(beta):
    sum_grad, sum_hessian = summed_matrices(beta)[0], summed_matrices(beta)[1]
    sum_hessian = sum_hessian.reshape(-1,2)
    return beta - np.linalg.inv(sum_hessian)@sum_grad

def bhcube_update(beta):
    sum_grad, sum_outer = summed_matrices(beta)[0], summed_matrices(beta)[2]
    sum_outer = sum_outer.reshape(-1,2)
    return beta + np.linalg.inv(sum_outer)@sum_grad

# (f) and (g)
print(newton_update([0,0]), bhcube_update([0,0]))

In [ ]:
# (h) for newton
def newton_loop(n):
    beta_newton = [0,0]
    iterations = 0
    while iterations <= n:
        update = newton_update(beta_newton)

        beta_newton = update
        iterations +=1
    return beta_newton

# (h) for bhcube
def bhcube_loop(n):
    beta_bhcube = [0,0]
    iterations = 0
    while iterations < n:
        update = bhcube_update(beta_bhcube)

        beta_bhcube = update
        iterations +=1
    return beta_bhcube
print(newton_loop(5), bhcube_loop(5))

In [ ]:
# MC simulation
B = 200
newton_estimates = np.zeros((B, 2))
bhcube_estimates = np.zeros((B, 2))

for b in range(B):
    X_b, Y_b = sampling(500, [1,1])[0], sampling(500, [1,1])[1]
    
    newton_estimates[b] = newton_loop(5)
    newton_estimates[b] = bhcube_loop(5)

mean_newton = np.mean(newton_estimates, axis=0)
std_newton = np.std(newton_estimates, axis=0)

mean_bhhh = np.mean(bhcube_estimates, axis=0)
std_bhhh = np.std(bhcube_estimates, axis=0)

print("Newton-Raphson Results: Mean:", mean_newton, "Std:", std_newton)
print("BHHH Results: Mean:", mean_bhhh, "Std:", std_bhhh)

In [ ]:
for i in [2, 10, 20]:
    print(str(i)+ ' iterations: ', newton_loop(i), bhcube_loop(i))

# conclusion: both estimates hover around the assumed population parameters